# Lecture overview


- Yield to maturity (YTM) and current yield


- Coupon reinvestment


- Realized bond yield (return)


- Bond realized return when interest rates change

In [1]:
# Load packages
import bond_pricing.simple_bonds as bp
import bond_pricing.present_value as pv
import pandas as pd

# 1. Yield to maturity and current yield

A bond's yield to maturity (YTM) is the rate that makes the present value of the bond's future cash flows equal to its current price. 

There is an **inverse relationship** between bond prices and their YTM: if a bond has a higher YTM, that means its cash flows are discounted at a higher rate, which means its price is lower. The implication is that bond prices go up when interest rates go down and they go down when interest rates go up.

A bond's **current yield** equals its annual coupon payment divided by the current bond price.

To calculate YTM using the "bond-pricing" package, we use the "bond_yield" function that requires the following parameters:

- mat = either maturity in years or date of maturity
- settle = either don't use it (if mat is expressed in years) or use date when you want to calculate the yield 
- price = **clean** price of the bond on the date when you want to calculate the yield
- cpn = annual coupon rate of the bond
- face = par value (aka face value) of the bond
- freq = number of compounding periods per year

#### Example 1:

Bond A matures in 5 years and it pays coupons semi-annualy. It has a coupon rate of 4% and a price of $1050. What is the YTM of the bond? What is the current yield of the bond?

In [2]:
# Calculate YTM
ytm = bp.bond_yield(mat = 5, freq = 2, cpn = 0.04, price = 1050, face = 1000)

# Print it out
print('bond YTM = ', ytm)

bond YTM =  0.02918011802922616


In [3]:
# Calculate current yield
cur_yield = (0.04 * 1000) / 1050

# Print it out
print("bond current yield = ", cur_yield)

bond current yield =  0.0380952380952381


#### Example 2:

Bond A was issued on 'June-01-2019' and it matures in 'Jun-01-2029'. Coupons are paid semi-annualy. The coupon rate is 8%. On Feb-01-2020, the dirty price of the bond is 1154.557 and the clean price is 1141.224 (the bond uses a '30/360' day count). What is the YTM of the bond on Feb-01-2020?

In [4]:
# Calculate YTM
ytm = bp.bond_yield(settle = '2020-02-01', mat = '2029-06-01', freq = 2,
                     cpn = 0.08, price = 1141.224, face = 1000, daycount = 'thirty/360')

# Print it out
print('YTM = ', ytm)

YTM =  0.06000007571287143


In [5]:
# Calculate current yield
cur_yield = (0.08*1000) / 1141.224
cur_yield

0.07010017314742768

# 2. Reinvesting coupons

Your return from a bond investment (see section 3 below) will depend on how you (re)invest the coupons. If you are able to reinvest your coupons at a higher rate, the total return from the bond will be higher. 

Below, we work through a couple of examples where we assume you are able to reinvest the coupons at a particular constant interest rate. This makes the coupons a fixed-rate annuity. The total proceeds from the coupons will therefore be the future value of that annuity. 

We can calculate future values of annuities with the "bond-pricing" package using the "annuity_fv" function in the "present_value" subpackage. The "annuity_fv" function requires the following parameters:

- rate = interest rate per compounding period 
- n_periods = number of coupons 
- instalment = coupon per compounding period (which may not be annual)


#### Example 3:

Bond A matures in 10 years and it pays interest quarterly. It has a coupon rate of 5% and a YTM of 10%. Calculate the total proceeds from the coupons of the bond if you reinvest all coupons at a 10% rate and you hold the bond until maturity.


In [6]:
# Inputs
T = 10
F = 4
crate = 0.05
ytm = 0.1
par = 1000
coupon = par * crate / F
coupon

12.5

In [7]:
# Price the bond today
price_when_bought = bp.bond_price(mat = T, freq = F, cpn = crate, yld = ytm, face = par)
price_when_bought

686.2153118489032

In [8]:
# Proceeds from coupons under 10% reinvestment rate
fv_coupons = pv.annuity_fv(rate = 0.1/F, n_periods = T*F, instalment = coupon)
print("Proceeds from coupons = ", fv_coupons)

Proceeds from coupons =  842.5319191949816


#### Example 4:

For the same bond as in Example 3 (matures in 10 years and it pays interest quarterly; it has a coupon rate of 5% and a YTM of 10%), calculate the total proceeds from the coupons of the bond if you do not reinvest them at all. 

In [9]:
# Proceeds from coupons under no reinvestment
fv_coupons0 = pv.annuity_fv(rate = 0 / F, n_periods = T * F, instalment = coupon)
fv_coupons0

500.0

# 3. Bond compounded realized yield (realized return)

The realized yield (or realized return) of a bond is your total return from investing in the bond. It equals the proceeds from the coupons (possibly reinvested) plus the price you obtain from selling the bond (or par value if you hold it until maturity), all divided by the price you paid for the bond initially.

Note that this give us a total, gross, compounded return on the bond. To convert it to an APR, we use the familiar conversion formula:

$$APR = F \cdot (Gross Compounded Return^{\frac{1}{T \cdot F}} - 1) $$

where

- Gross Compounded Return = bond total proceeds (including reinvested coupons) divided by price paid for bond
- T is number of years the bond was held, and F is the number of coupons (or compounding periods) per year

#### Example 5:

For the same bond as in Example 3 (matures in 10 years and it pays interest quarterly; it has a coupon rate of 5% and a YTM of 10%), calculate the realized return of the bond if coupons are reinvested at a 10% rate and the bond is sold at maturity. 

In [10]:
# Total proceeds from bond under 10% coupon reinvestment rate
fv_total = par + pv.annuity_fv(rate = 0.1/F, n_periods = T*F, instalment = coupon)
print("Total proceeds from bond = ", fv_total)

Total proceeds from bond =  1842.5319191949816


In [11]:
# Total (gross) realized return of the bond under 10% coupon reinvestment rate
hpr = fv_total / price_when_bought
print("Total HPR = ", hpr)

Total HPR =  2.685063838389963


In [12]:
# Annualized realized return under 10% coupon reinvestment rate (we are converting an EAR into an APR)
hpr_annual = (hpr ** (1/(T*F)) - 1) * F
print(f'Bond realized return under 10% coupon reinvestment = {hpr_annual: .3f}')

Bond realized return under 10% coupon reinvestment =  0.100


#### Example 6:

For the same bond as in Example 3 (matures in 10 years and it pays interest quarterly; it has a coupon rate of 5% and a YTM of 10%), calculate the realized return of the bond if coupons are not reinvested and the bond is sold at maturity. 

In [16]:
fv_total0 = par + pv.annuity_fv(rate = 0/F, n_periods = T*F, instalment = coupon)
print("Total proceeds from bond = ", fv_total0)

hpr0 = fv_total0 / price_when_bought
print("Total HPR = ", hpr0)

hpr_annual = (hpr ** (1/(T*F)) - 1) * F
print(f'Bond realized return under 10% coupon reinvestment = {hpr_annual0: .3f}')

Total proceeds from bond =  1500.0
Total HPR =  2.185902841425204
Bond realized return under 10% coupon reinvestment =  0.079


# 4. Realized returns when interest rates change

It is important to note that, if interest rates change after you buy a bond, both your reinvestment rate and your YTM will likely change in the same direction. If rates drop, this is bad news for the value of the coupon payments (since you will be reinvesting them at a lower rate) but it is good news for the resale value of the bond itself (since a lower YTM means a higher price). The opposite happens if interest rates increase after you purchase the bond.

#### Example 7:

Bond A matures in 10 years and it pays interest quarterly. It has a coupon rate of 5% and a YTM of 10%. You buy the bond today, and over the next 4 years, you reinvest all coupons at an 8% rate. In 4 years, the bond's YTM is 7% and you sell it. What was your realized return from your investment in this bond?

In [13]:
# Proceeds from coupons
fv_coup = pv.annuity_fv(rate = 0.08/F, n_periods = 4*F, instalment = coupon)

# Proceeds from sale of bond
sale_p = bp.bond_price(mat = 6, freq = F, cpn = crate, yld = 0.07, face = par)

# Total HPR
hpr = (fv_coup + sale_p) / price_when_bought

# Annual realized return
hpr_annual = (hpr ** (1/(4*F)) - 1) * F
print(f'Bond realized return  = {hpr_annual: .3f}')  

Bond realized return  =  0.128


#### Example 8:

Bond A has a YTM of 6% and a coupon rate of 4%. It has a face value of \\$100, it matures in 8 years and pays coupons annually. Suppose you buy the bond today and reinvest all coupons at a rate of 4\%. You sell the bond in 6 years. At the time of sale, the YTM of the bond was 8\%. Calculate the realized return over the 6-year holding period (expressed as an APR).  

In [17]:
ytm = 0.06
cr = 0.04
par = 100
ttm = 8
f = 1
reinv = 0.04
new_ytm = 0.08
holding_period = 6

coupon = par * cr / f
nr_payments = holding_period * f
ttm_at_sale = ttm - holding_period

print(coupon, nr_payments, ttm_at_sale)

4.0 6 2


In [18]:
price_when_bought = bp.bond_price(mat = ttm, freq = f, cpn = cr, yld = ytm, face = par)
price_when_sold = bp.bond_price(mat = ttm_at_sale, freq = f, cpn = cr, yld = new_ytm, face = par)

print(price_when_bought, price_when_sold)

87.58041237806088 92.86694101508915


In [19]:
fv_coup = pv.annuity_fv(rate = cr/f, n_periods = nr_payments, instalment = coupon)
fv_coup

26.531901849600036

In [21]:
hpr = (fv_coup + price_when_sold) / price_when_bought
hpr

hpr_annual = (hpr ** (1/(holding_period * f )) - 1) * f
hpr_annual

0.05300925605120432

# 5. Resources


- bond-pricing package
    - https://bond-pricing.readthedocs.io/en/latest/
    
    
- isda_daycounters
    - https://github.com/miradulo/isda_daycounters

